In [ ]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


url = "https://raw.githubusercontent.com/CharlesCLuo/Application-of-AI-in-Supply-Chain-Risk-Management-Series/main/Demand_Forecsting/demand_data.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')


data_normalized = data.copy()


scaler = MinMaxScaler()
data_normalized[['Demand']] = scaler.fit_transform(data[['Demand']])


train_size = int(len(data_normalized) * 0.8)
train_normalized, test_normalized = data_normalized.iloc[:train_size], data_normalized.iloc[train_size:]


X_train, y_train = train_normalized[['Demand']].values, train_normalized['Demand'].values
X_test, y_test = test_normalized[['Demand']].values, test_normalized['Demand'].values


X_train = X_train.reshape((X_train.shape[0], 1, 1))
X_test = X_test.reshape((X_test.shape[0], 1, 1))


model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1)))
model.add(Dropout(0.2))
model.add(Dense(1))


model.compile(optimizer='adam', loss='mean_squared_error')


history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=1)


y_pred_normalized = model.predict(X_test)


y_pred_rescaled = scaler.inverse_transform(y_pred_normalized.reshape(-1, 1))
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))


plt.figure(figsize=(14, 7))
plt.plot(data.index[train_size:], y_test_rescaled, label='Actual Demand (Test Data)', color='orange', linestyle='--')
plt.plot(data.index[train_size:], y_pred_rescaled, label='LSTM Forecasted Demand', color='red', linestyle='-')
plt.axvline(x=data.index[train_size], color='black', linestyle=':', linewidth=1.5, label='Train-Test Split')
plt.title('Figure 3.4 LSTM Model - Actual vs. Forecasted Demand')
plt.xlabel('Date')
plt.ylabel('Demand')
plt.legend()
plt.grid(True)
plt.show()


mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
mape = mean_absolute_percentage_error(y_test_rescaled, y_pred_rescaled) * 100


print(f"LSTM Model - MAE (original scale): {mae:.2f}")
print(f"LSTM Model - RMSE (original scale): {rmse:.2f}")
print(f"LSTM Model - MAPE (original scale): {mape:.2f}%")
